<a href="https://colab.research.google.com/github/minkyoJang/AI-BigDataStudy/blob/master/jmkDay24_Seq2Seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Copyright(C). Cheonbok Park. All rights reserved.**

Email : cb_park@korea.ac.kr

### 필요 라이브러리 설치

In [0]:
# 필요한 라이브러리를 설치합니다. 

In [0]:
import torch # torch library 
import torch.nn as nn # Nueral Network에 대한 package
import numpy as np  # numpy 
import editdistance # 평가 지표로서 사용될 edit distance 
import matplotlib.pyplot as plt # plot 을 찍기 위한 라이브러리
import tqdm
import torch.nn.functional as F # pytorch function 들을 사용하기 위한 용도 
from torch.utils import data # dataset 관련된 utility 를 사용하려는 용도
from random import choice, randrange # random
from itertools import zip_longest 
import librosa
import os   # directory 생성 및 디렉토리 생성과 관련된 package 
import json 
import random


### Data Loader 
- 문제점 있어요. NLP에서 가질 수 밖에 없음
- 원핫인코딩말고 I go to the school이라고 하면 i 가 토큰이 1번 인덱스다라해봐요
- 그럼 0 1 0 0 넣지 않고, i를 그냥 1이라고 하고 go는 7번, to는 21, the 는 3, school50번째 토큰으로 하나의 인덱스 할 수 있음 (토큰 5개)
- 한 문장만 트레이닝하지 않고 하나의 배치에 여러 문장 들어갈 것
- 그럼  i have to rend the book 이라는 문장에 대해서도 토큰이 6개들어가.
- 지금 이 두 문장은 하나는 5토큰, 하나는 6토큰. 이거 매트릭스 불가
- 즉 이 문제를 해결해주려면 패딩을 해줄 수 밖에 없음.  패딩 중요**

#### 길이?
- 샌탠스에 배치 안에서 최대 길이에 맞춰줘야합니다.
- 만들어주지 않으면 매트릭스 자체가 안되어 연산 깨짐

![대체 텍스트](https://pytorch.org/tutorials/_images/word-encoding.png)

In [0]:
def batch(iterable, n=1):
    args = [iter(iterable)] * n
    return zip_longest(*args)

# 패드 토큰은 거의 0으로 넣습니다.
def pad_tensor(vec, pad, value=0, dim=0):
    """
    pad token으로 채우는 용도 
    args:
        vec - tensor to pad
        pad - the size to pad to
        dim - dimension to pad
    return:
        a new tensor padded to 'pad' in dimension 'dim'
    """
    pad_size = pad - vec.shape[0]

    if len(vec.shape) == 2:
        zeros = torch.ones((pad_size, vec.shape[-1])) * value
    elif len(vec.shape) == 1:
        zeros = torch.ones((pad_size,)) * value
    else:
        raise NotImplementedError
    return torch.cat([torch.Tensor(vec), zeros], dim=dim)


#### 아래 그림 설명 padded sequence

- RNN을 만들었어요 아래처럼
- - RNN 넣을때 이렇게 넣으면 빠르게 딥러닝 돌아가서 파이토치 지원
- 맨 아래부터 2,3,4,5,..개의 토큰이 있어요
- 이거 하나/둘/셋/넨/다/여섯. 이렇게 할때 패드토큰까지 주면 좋을까요?
    - 얘네 히든도 있고 시퀀스도 넣어주고 하면.?
    - 지금 메모리 낭비하고 있음. 
    - 해당되는 애들을 길이나 시퀀스 고려해서 뱇 사이즈를 다이나믹하게 늘렸다줄일수 있음
    - 배치 사이즈를 처음엔 6개로 하고, 두번쨰는 전체 배칠아 같은 6개 넣고, 세번쨰 시퀀스부터는 하나 빼줘. 이제 업데이트 안해. 그리고 네번째 시퀀스 부터도 하나씩 빼. 그럼 속도 빨라질 수 있겠지. 낭비하는 메모리가 없으니까. 이게 다이나믹하게 하기 위한 시작 **
    - 파이토치는 이런거 귀찮아 rnn은.
    - cnn 플젝트하면 쉬운데 rnn은 장벽 있어. 프리프로세싱 고려해서 모델을 만들어야
- 즉 시퀀스 나눠줘야함
    - 저렇게 해서 rnn hidden statte

![대체 텍스트](https://i.stack.imgur.com/Kuhh0.jpg)

In [3]:
def pad_collate(batch, values=(0, 0), dim=0):
    """
    데이터 로더에 들어가기전에 batch화 할 때 거치는 함수 
    args:
        batch - list of (tensor, label)
    reutrn:
        xs - a tensor of all examples in 'batch' after padding
        ys - a LongTensor of all labels in batch
        ws - a tensor of sequence lengths
        
        
        #데이터 로더
- 파이토치점 데이터로더 적용은 일단은 1~10까지 데이터가 있다 생각
- 랜덤하게 셔플링하여 순서 잘 바꿔서 2, 7, 6이 나왔어요. 그럼 2,7,6번째 인덱스 것만 가져올거에요.
- [data]-len->[dataloader0~9]--2,7,6  뽑아요-- 미니배치 사이즈
>> [data]에서 getitem해서 (2,7,6)으로 하나씩 뽑아와서 sentesce1, s2,s3 을 가져옵니다. (얘네 아직 매트릭스 합치기 전)
	>> 데이터 로더는 뽑기만 하거 어찌 합치는 지몰라ㅣ
	>> 그래서 s1,s2,s3 합치는 함수가 nlp에서 필요해요 
		>> s1,s2,s3를 하나의 매트릭스로 만드는 함수. 
** nlp_pad_collate***
		->이미지는 사실 필요없어요. 디맨젼이 같으면 자동으로 concat되니까
		>> 그러나 nlp는 길이가 달라서 merge할 함수 ㅠㅣㄹ요하고 이게 pad-collate
			>> 최대 길이 뽑고 그 중에서 부족한건 패드
			>> 소스가 가장 큰 길이가 맨위에, 작을 수록 아래에 두는 역할 합니다.
            
            
    """

    sequence_lengths = torch.Tensor([int(x[0].shape[dim]) for x in batch]) # 각 batch 마다 길이를 얻어내고 
    sequence_lengths, xids = sequence_lengths.sort(descending=True) # 감소하는 순서로 정렬. xids 디센딩했을때 순서. 
    target_lengths = torch.Tensor([int(x[1].shape[dim]) for x in batch])
    # find longest sequence (가장 긴 sequence의 길이를 구함 )
    src_max_len = max(map(lambda x: x[0].shape[dim], batch))  
    tgt_max_len = max(map(lambda x: x[1].shape[dim], batch))
    # pad according to max_len (max length 만큼 padd를 추가 )
    batch = [(pad_tensor(x, pad=src_max_len, dim=dim), pad_tensor(y, pad=tgt_max_len, dim=dim)) for (x, y) in batch]

    # stack all #length가 위의 과정으로 다 같으니까 이제 스택을 해줘요. 근데 아직은 디센딩 오더로 정렬 전. 
    xs = torch.stack([x[0] for x in batch], dim=0)
    ys = torch.stack([x[1] for x in batch], dim=0)
    xs = xs[xids].contiguous() # decreasing order로 다시 나열(xids 디샌딩으로 정렬되어있으니 이거 참조해서 함. ) 
    ys = ys[???].contiguous() # xids 와 같은 순서로 . 두번째있는 시퀀스 올리면 소스 따라 타겟도 올라가야 페어가 맞죠***@***
    target_lengths = target_lengths[???] # 마찬가지로 페어 맞춰야 하니까 xids. same descending order
    return xs.long(), ys.????(), sequence_lengths.int(), target_lengths.int() #워드 임배딩으로 인식되려면 롱으로 해야해. 그래서 토큰을 롱으로 자형 선언 필요


class ToyDataset(data.Dataset): #1.데이터 셋 상속해야 여러 장점 있어요. 2가지 아이템 필요(랜, 겟 아이템)렌은 데이터 셋의 전체 랭쓰 3만개라면 이 정보, 겟 아이템은 데이터셋의 첫번쨰 인덱스 두번째 데이터 등
    """
    https://talbaumel.github.io/blog/attention/
    """
    def __init__(self, min_length=5, max_length=20, type='train'):
        self.SOS = "<s>"  # all strings will end with the End Of String token )
        self.EOS = "</s>"  # all strings will end with the End Of String token
        self.characters = list("abcde")
        self.int2char = list(self.characters)
        self.char2int = {c: i+3 for i, c in enumerate(self.characters)} # +3 을 왜하는 가?
        print(self.char2int)
        self.VOCAB_SIZE = len(self.characters)
        self.min_length = min_length
        self.max_length = max_length
        
        # train set or test set 을 생성 
        if type == 'train':
            self.set = [self._sample() for _ in range(4000)]
        else:
            self.set = [self._sample() for _ in range(300)]

    def __len__(self):
        # 필수 ! 
        return len(self.set)

    def __getitem__(self, item):
        # 필수 !
        return self.set[item]

    def _sample(self): #안중요하니 패스
        random_length = randrange(self.min_length, self.max_length)  # Pick a random length
        random_char_list = [choice(self.characters[:-1]) for _ in range(random_length)]  # Pick random chars
        random_string = ''.join(random_char_list)
        a = np.array([self.char2int.get(x) for x in random_string]+[2])
        b = np.array([self.char2int.get(x) for x in random_string[::-1]] + [2]) # Return the random string and its reverse + EOS 
        
        return a, b


SyntaxError: ignored

### utils misc.py
- 추후 설명



In [0]:
EOS_TOKEN = '</s>'


def check_size(tensor, *args):
    size = [a for a in args]
    assert tensor.size() == torch.Size(size), tensor.size()

def to_mono(y):
    assert y.ndim == 2
    return np.mean(y, axis=1)


def edit_distance(guess, truth):
    guess = guess.split(EOS_TOKEN)[0]
    truth = truth[3:].split(EOS_TOKEN)[0]
    return editdistance.eval(guess, truth) / len(truth)


class AttrDict(dict):
  __getattr__ = dict.__getitem__
  __setattr__ = dict.__setitem__

### Edit distance (편집거리 알고리즘) 

![대체 텍스트](https://raw.githubusercontent.com/sumitc91/data/master/askgif-blog/9e07d056-ccf7-4fc8-b6ee-000c8032b9ec_editDistance.gif)

In [0]:
# edit distance 란 편집 거리 


ref = [1, 2, 3, 4]
hyp = [1, 2, 4, 5, 6]
editdistance.eval(ref,hyp)

### Attention Mask
- 추후 설명

In [0]:
8## 추후에 설명 Decoder section
def mask_3d(inputs, seq_len, mask_value=0.):
    batches = inputs.size()[0]
    assert batches == len(seq_len) # length 체크 
    max_idx = max(seq_len) # max length 체크 
    for n, idx in enumerate(seq_len): # length 에서 의미없는 hidden state attention 값은 0으로 두기 위한 mask값 설정 
        if idx < max_idx.item():
            if len(inputs.size()) == 3:
                inputs[n, idx.int():, :] = mask_value
            else:
                assert len(inputs.size()) == 2, "The size of inputs must be 2 or 3, received {}".format(inputs.size())
                inputs[n, idx.int():] = mask_value
    return inputs

## Encoder RNN

![대체 텍스트](https://pytorch.org/tutorials/_images/encoder-network.png)

# 텐서플로
- 컴퓨테이션 그래프
- 딥러닝들은 연산하는 플로우가 있고 이거 블록으로 조립하는 식
- 파이토치나 텐서플로는 그런 그래프 그리고 백프로파게이션 연산하는 식.
- 코딩할때 먼저 컴퓨테이션 그래프 그리는게 첫 시작이에요
- 그게 ㅇ위에 있는 것


### 저런 그래프 어떻게 만드나요?
- 모델을 그려보면서 하면 됩니다. 아래 그림을 볼까요

#### Embedding Module 
- 보캐블러리 넘버
- 임대빙 딤
- 패딩 인덱스: 영번 은 패드 토크니라고 해서 의미가 없다고 했죠. 얘네 임베딩은 다 0으로 채움. 넘버 주면 알아서 학습 안하고 0 뱉음


#### gru함수
- 3가지 학스ㅏㅂ하게 할거야
- 인풋 정하면 나오고 바이디랙션도 하게해
- 많은 파라미텅 ㅣㅆ음
    - 인풋 사이즈
    - 히든 사이즈: 그거 안에 있는 히든 스테이터스 수
    - 넘버 오브 레이어: 멀티 레이어 얼마나 쌓을 건지에 대한 제약조건
    - 배치 퍼스트: 기본적으로 트루를 주곤 합니다.
    - 인풋을 어떻게 줄 수 있냐면 일반적으로 (batch, 3 , w, h)였을거에요 rgb는
    - 그리고 지금은 (B,T)하고 아니면 (T,B) 모델은 타임스컨스로 보기에 T를 먼저 생각
        >> b,t 순서면 배치 퍼스트 트루로 줘야해 *********@@*******
        >> 드랍아웃 확률값 주려면 확률값 주려고
    - 바이디렉션: 각 디맨션 마다 추가적인 하나의 백워드 방향의 gru 생성

![대체 텍스트](https://i.ibb.co/S0NJzq7/embedding.png)

#### GRU Module

![대체 텍스트](https://i.ibb.co/881BygH/GRU.png)

![대체 텍스트](https://i.ibb.co/NsMqvcH/GRU-param.png)

#### ENCODER RNN Code
- 엔엔점 모듈 먼저 행해. 그래야 뉴럴네트워크인거알지
- 인잇은 무조건 레고 선언. 여기서는 2개 필요해요. 임베딩과 알엔엔. 그럼 이거에 대해 무조건 선언해줘야한다고 생각하면돼

- 1. 우선 임베딩 선언해요 nn.embedding 그리고
- n classes는 아까 말했던 넘버오브 클래스 갯수. 캐릭터의 갯수 가져오는게 config.get
- self.embedding_dim 넣어서 (?????) 몇개 임베딩 하는지 얘기
- 패드 토큰 0을 넣어서 앞으로 얘는 패드 토큰을 영 쓴다 말해요. 

-2. 지알유의 인풋
- 지알유인풋디맨션은 임베딩의 디맨션이 되어요. ** (self.embedding_dim)
- 알엔엔 선언해야하고, 그 중에서 지알유 선언해요. 그리고 엘에스티엠하고 싶음 대문자로 쓰고 (GRU)

-3. 지피유 트루하고 각종 지알유 선언한 것들 해봐요


-4 포워드로 레고 순서 나열
- 인풋은 처음에 어디 지나냐면 임베딩 지나야 해요. 그러면 (self.embedding)
- RNN에 얘를 넣어야 하죠 이떄 패딩 시퀀스를 먼저 해줘요 ** 다이나믹하게 시퀀스 만들려면 팩 패딩 시퀀스로 해(인풋, 걔네 각각 실제 길이 담고, 배치퍼스트는 비,티 순서라서 트루로줍니다.)

-5. 이렇게 배킹이 된 애를 엑스에 넣고, 히든은 프리비어스 히든 스테이트인데 여기선 이니셜

-6. 일단 아웃풋과 스테이터스 나오고, 아웃풋은 마찬가지로 의미있는 시퀀스만 저장해둬서 현재 패킹된 상태임. 그래서 이 output을 pad_packed_squence로 풀어줘야합니다.

//얼굴보니 모르겟는거 같아 설명
배치 사이즈 RNN 돌리게 되면 아웃풋은 이걸 줘요. 각각 타임 스탭에 대한 정보를 주는게 결과를 주는게 아웃풋이에요. 그러니까 마지막 레이어에 각 토큰 마다의 히든 스테이트 주는게 아웃풋 **. 그 다음에 스테이트는 마지막 시퀀스의 각 레이어별 히든 스테이터스 주는 것. 이걸 줘야 나중에 디코더 초기로 넣어야하니까. 그래서 가지고 있어야하죠.
- 일단 아웃풋이 있고 아웃풋은 4개의 토큰이 있다고 하면 아웃풋은 4곱하기 히든 사이즈만큼 있을 거에요
- 이제 배치가 예를 들어서 3개가 있다면 삼곱하기 사곱하기 히든 사이즈.
- 이렇지 않은게 배치를 다 하지 않죠. 그래서 우리가 첫번쨰 시퀀스에서 이런게 배치가 6개마다 있고, 둡너째도 6개 있지만, 세번째는 5개 있죠. 
- 그럼 이빨이빠진 부분이 있어서 텐서가아니게 되는데 이때 는 전체 3이고 4해서 히든사이즈 곱해주는거에요. 이 애들이 음. (현재 무지개 색있는 시퀀스가지고도 설명)
- 지금 우리가 패킹을 안 풀어주면 전체 데이터는 9개가 있을거엥 시퀀스. 이거 패킹 푸면 3개를 빵빵 의미없는 매트릭스로 채워줘요. 그래서 총 삼곱하기 사의 히든사이즈가 있죠. 거기서 ㅇ이콤마 삼, 이콤마 이 의 위치는 다 빵이겠죠. 
- 원래 매트릭스 사이즈 복구를 위해서 보는 pad packed. 
    - 패킹을 할때 팩 패크드이고, 언패킹 할떄는 바꺼서 이야기하면 돼. 
- 7. 바이 디렉션의경우에는 콘켓도 되지만 일단 섬을 할 수 있어서 섬을 할게요(콘켓이 맞긴함)(self.hiddensize)

-8. 반반 쪼개서 해줘요. 바이디렉션은 앞에서 부터, 뒤에서 부터하니까. 

-9. 인잇 히든 스테이터스
- 아무것도 없으면 영으로 넣으니 (.zeros) 넣어줍니다.

In [5]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
class EncoderRNN(nn.Module):
    def __init__(self, config):
        super(EncoderRNN, self).__init__()
        self.input_size = config["n_channels"]
        self.hidden_size = config["encoder_hidden"]
        self.layers = config.get("encoder_layers", 1)
        
        self.dropout = config.get("encoder_dropout", 0.) 
        self.bi = config.get("bidirectional_encoder", False)
        embedding_dim = config.get("embedding_dim", None)
        self.embedding_dim = embedding_dim if embedding_dim is not None else self.hidden_size
        self.embedding = nn.???(config.get("n_classes", 32), ????, padding_idx=??)
        gru_input_dim = ????
        self.rnn = nn.????(
            gru_input_dim,
            self.hidden_size,
            self.layers,
            dropout=self.dropout,
            bidirectional=self.bi,
            batch_first=True)# model 선언 
        self.gpu = config.get("gpu", False) 



    def forward(self, inputs, hidden, input_lengths):
        
        # pack padded 를 통하여 input을 감싸기 
        inputs = self.????(inputs)
        
        x = pack_padded_sequence(inputs, input_lengths, batch_first=True) # ??? 무엇일까?
        output, state = self.rnn(x, hidden)
        output, _ = pad_packed_sequence(output, batch_first=True, padding_value=0.) # sequence 를 위의 그림과 같이 pack함 
        
        if self.bi: # bidirectional 의 경우 forward와 backward를 sum하여 사용한다. or concat 
            output = output[:, :, :self.????] ??? output[:, :, self.????:]
            state = state[:1] +state[1:]
        return output, state

    def init_hidden(self, batch_size):
        # hidden state가 없는 초기 상태일때 
        h0 = torch.???(2 if self.bi else 1, batch_size, self.hidden_size)
        if self.gpu:
            h0 = h0.cuda()
        return h0

SyntaxError: ignored

![대체 텍스트](https://pytorch.org/tutorials/_images/attention-decoder-network.png)

# 디코드

- 빨간색은 디코드 알엔엔이 있어요(빨강)
- 이거 바탕으로 어텐션을 해야하죠. 어탠션을 해주게 되는 어탠션 모듈이 필요해요. (핑크)
- 디코드코드 아래에 있어야 하는건 [임배딩][알엔엔 층]
- 디코드이 토큰이 들어가야하고
- 인코드에서 마지막에 생긴 히든 스테이터스가 디코드에서 들어가는 용이겠죠
- 인코드의 라스트 히든 스테이터스 가 저장이 되고 
- 핑크가 만들어넨 ct와 rnn의 쿼리 스테이터스를 concoat하고 리니어 레이어 거쳐야겠죠
- 리니어 레이어 거치면 각각 보캡에 대한 확률 구할 수 있죠. ㄱ래서 각 보캡 수만큼 확률 값이나올거에요(=로짓값)

#### 어탠션 모듈 필요한 거
- 히든 스테이터스 필요
-참조 메몰 ㅣ필요(인코더 히든스테이터스들)
- ht랑 비교해줘야해
- dot 프로덕트ㅡㄹ 이제 할건데 우리의 소스가 바로 인코더쪽의 히든 스테이터스와 타겟인 디코드 쪽의 히든 스테이트를 dot 프로덕트할거에요. 그럼 어떤 스칼라 값이 나올거에요
- 그게 바로 어텐션의 로짓이고 소프트맥스 거쳐서 웨이티드 에브리지하면 결과 나와요
-------------------------------------------------

1. 디코드에 필요한거 우선 지알유
2. (디코드에 바이디랙션해도 될까요? 안돼. 왜냐하면 우리가 바이디렉션 하면 치팅을 하는거죠. sos이고 nan이 들어가면 바이디렉션 하는 순간 난의 정보가 위에 올라가니 sos가 치팅할 수 있어. 그래서 바이디랙션은하면 안돼******그래서 어느 경우에도 false)
3. 아..루옹은 어텐션 거쳐서 하는데, 반하은은 어탠션자체가 어.....
- 그림 잘못 가져오셨대
-  히든 스테이터스> 어탠션> c> 리니어레이어에서 예측하는 루옹. 
    - 현재 필요한 정보를 꺼내
- 바안) 내가 필요한 다음거 가져온다
- 즉 rnn돌려서 내가 필요한 컨탠츠를 가져와서 concat해서 쓰는 루옹과, 일단은 rnn돌리고 힏느 스테이터스 어탠션해서 그 결과를 얘의 인풋으로 넣어. 
- 즉 루옹은 지금 필요한거 가져오는 식이고, 반항은 다음번에 필요한걸 가져오는 식. 그래서 어탠션을 만든 context 벡터가 rnn 인풋으로서 concat해서 들어가는게 오른쪽 반흥 내용.
- 루옹과 반흥. 코드보면 어탠션 모델이 제ㅐ각각이긴해. 어떤건 이렇고 어떤건 저래. 이미지 캡셔닝은 bahdanau같아. 초기에서 어탠션 가져와서 인풋과 concat하고 그 히든 을 가지고 다음 콘태스트 뭐 넣을지는 인풋으로 넣었지. 그래서 이미지는 무엇을 볼지 알려주는거고
- 루옹은 지금 필요한 정볼르 가져오는 모델들 

- 실습 코드는 반흥거 가져왔네요ㅜㅜ

### 디코드
- 1. 지알유에 false
    - 일단은 프리비어스 히든 스테이터스 기반으로 어탠션을 구하고 그 어탠션 깁나으로 소프ㅡ 맥스한걸 인코더의 아웃풋과 계싼해서 가져오고 그걸 기반으로 gru에 콘캣으로 준다는거
    
-2. 디코더 선언3가지(어탠션 모델, rnn, 임배딩)
- nn.embedding(패딩 인댁스도 넣어줌)
- nn.GRU
- attention

그래서 이건 따로 만ㄷ르어줘야하고 그럼 디코더에 대한 추상 클래스 완성

### Decoder 

In [0]:
class Decoder(nn.Module):
    def __init__(self, config):
        super(Decoder, self).__init__()
        self.batch_size = config["batch_size"]
        self.hidden_size = config["decoder_hidden"]
        embedding_dim = config.get("embedding_dim", None)
        self.embedding_dim = embedding_dim if embedding_dim is not None else self.hidden_size
        self.embedding = nn.Embedding(config.get("n_classes", 32), self.embedding_dim, ???=0)
        self.rnn = nn.???(
            input_size=self.embedding_dim+self.hidden_size if config['decoder'].lower() == 'bahdanau' else self.embedding_dim,
            hidden_size=self.hidden_size,
            num_layers=config.get("decoder_layers", 1),
            dropout=config.get("decoder_dropout", 0),
            bidirectional=????,
            batch_first=True)
        if config['decoder'] != "RNN":
            self.attention = Attention(
                self.batch_size,
                self.hidden_size,
                method=config.get("attention_score", "dot"))

        self.gpu = config.get("gpu", False)
        self.decoder_output_fn = F.log_softmax if config.get('loss', 'NLL') == 'NLL' else None

    def forward(self, **kwargs):
        """ Must be overrided """
        raise NotImplementedError

![대체 텍스트](https://i.ibb.co/gvpn1RT/bmm.png)

####
- 루옹 
     - 파란색 : 인코드. 우선적으로 만들어요. 
          - 인코드 들어가기전에 인풋을 임베딩으로 바꿔줘야해요(벡터화해야 연산 가능)
            - 그래서 파란색 하단에는 embedding 레이어가 존재해야해요
           - 현재 RNN 레이어 2개 쌓아야 하니 임베딩 위에 [RNN]을 두개 올려요
            - 바이디렉션 항상 고민해야해요. 여기서는 오케이
            - 이제 인코더로 나온게 '히든 스테이트'
        - 인코더의 인풋?
               - 초기 히든 스테이트스
        
    - 히든 스테이트스
    
    ----------------------------
1. 초기 인풋 고려
     > 현재 토큰화 되었으니 임배딩 거쳐야해요 [인풋]>[임배딩]
            >> 앤드 투 앤드: 원하는 인풋에서 원하는 결과가 나오게 해서 우리 인풋과 아웃풋만 넣으면 되는것
            >>얘를 통해 색을 반쪽 만들고 앤드투 앤드가 아닌게 중간에 무슨 연산 넣는것
2. 히든 스테이트스(피리비어스 히든 스테이터스)

[input> (EMBEDDING+Prev_hidden)> RNN> encode state] 
    

![대체 텍스트](http://cnyah.com/2017/08/01/attention-variants/attention-mechanisms.png)



```
# 코드로 형식 지정됨
```
# 어탠션
- 어탠션이 제일 중요하겠죠
- 어탠션(포어드 함수)
    - 프리비어스 히든 스테이터스 줘야해. 이게 쿼리로 들어가니까
    - 인코드 히든 스테이터스
    - 그거에 대한 시퀀스까지 주면 어탠션 계싼해서 weight 나와
        - 웨이트는 배치의 갯수만큼 있어서 B*T만큼 있어 맥스 시퀀스만큼(패드는 0)
    - context= > > b i t인데 b t h를 해야하네요.
        - 이제 우리는 각각 곱해줘서 더해야하는데 가장 쉬은 방식이 포문같죠?
            - 모델 안에서 포문쓰면 속도 느려져서 매트릭스 계싼해야해***
                - 배치 매트리스 멀티 플랫(B,a,b)디맨션과 (B,b,c)있으면 0번째/1번째/2번째에 대해서 매트릭스 ....를 해서 결과적으로 (B,a,c)나오게 한대@@아하 b는 겹치니까 그거 빼고 했네
                - 이렇게 하기 위해서 배ㅣ 매트리스 멀티 플랙션
 - 언스퀴즈
    - 첫 디맨션 생기게 하는것
    - 그러면 b t가 나중에 b , 1, t가 되는거에요. [B, T, H]-> [B, 1 H ]
        - 콘테스트에서 bnn하면 뭐고..  
        
   - 스퀴즈 : 해당 디맨젼 없애기
    - 그래서 [B, H로 줄임]
    - 이 부분까지 하면 context 벡터까지 한거에요 
    
  - 이제 왼쪽의 인풋을 임베딩을 해줘야 해서얻을 수 있고
  - 어텐션으로 얻은걸 추가해서 인풋을 제공합니다. rnn_input(context_vector)언스퀴즈하면돼. 그러면 (B Embedding> 1, b emeddin되고 이 상태에서 앞에 1을 맞춰야 해서 콘캣하면 1,b , embeding> 1,b hiddensize 되어 결국 1,b, hidden이 되는거래요. )

- 콘캣하는 연산
    - torch.cat
    
- 이제 rnn 넣어
- 트랜스폼 하는 이유: 배치 퍼스트
    - (1,b,emb+h)> 1이 앞으로 가고 0번째가 뒤로 간다고 해서 트랜스
        >> transpose(1,0)> (b,1,emb+h)
  - 린리어 레이어 . 
    - 원하는 아웃풋사이즈가 되어야 하니까 똑같이
    - self.output_size
    
 - 아까 아웃풋이 (b,1,hidden)인데 이거 맞춰서 (b,logits)되게 하여 원하는대로 되게 합니다. 

In [0]:
    class BahdanauDecoder(Decoder):
    """
        Corresponds to BahdanauAttnDecoderRNN in Pytorch tutorial
    """

    def __init__(self, config):
        super(BahdanauDecoder, self).__init__(config)
        self.output_size = config.get("n_classes", 32)
        self.character_distribution = nn.Linear(self.hidden_size, self.???)

    def forward(self, **kwargs):
        """
        :param input: [B]
        :param prev_context: [B, H]
        :param prev_hidden: [B, H]
        :param encoder_outputs: [B, T, H]
        :return: output (B), context (B, H), prev_hidden (B, H), weights (B, T)
        """
 
        input = kwargs["input"] # decoder input 
        prev_hidden = kwargs["prev_hidden"] # decoder rnn 에서 들어갈 previous hidden state 
        encoder_outputs = kwargs["encoder_outputs"] # encoder RNN에서 Encoding이 끝난 (B,L,hidden_size)  
        seq_len = kwargs.get("seq_len", None) # sequence length 

        # check inputs
        
       

        # Attention weights
        weights = self.attention.forward(prev_hidden, encoder_outputs, seq_len)  # B x T
        context = weights.unsqueeze(1).???(encoder_outputs).squeeze(1)  #[B,1,T] [B,T,H] -> [B x H]

        # embed characters
        embedded = self.embedding(input).unsqueeze(0)
        
        #attention 을 통해 얻어낸 context를 추가하여 모델에 input으로 제공
        rnn_input = torch.???((embedded, ???.unsqueeze(0)), 2)

        outputs, hidden = self.rnn(rnn_input.transpose(1, 0), prev_hidden.unsqueeze(0)) # 1 x B x N, B x N

        
        output = self.????(outputs.squeeze(0)) # logit 값 각 chracter 별로

        if self.decoder_output_fn:
            # NLL loss 인 경우 
            output = self.decoder_output_fn(output, -1)

        if len(output.size()) == 3:
            output = output.squeeze(1)

        return output, hidden.squeeze(0), weights


![대체 텍스트](https://i.stack.imgur.com/tiQkz.png)

# 어텐션
- 제너럴 닷 프로덕트
    - 제너럴: 바이미니얼. 인풋에 대한 디맨션을 참조로 써줘야해요. 
        - 임베딩 및 디맨션 일치하고 옆에 표기해줘야해요.
        - 라스트 히든을 배치*히든, 아웃은 배치사이즈, 맥스, 히든할거니까  저렇게 줙씀
     - 각각 파라미터 정의 되고 추가된거 필요없으니 두고, 히든 사이즈 투 히든 사이즈. 
     - concat은 2개 파라미터 필요하니까 히든사이즈 2배가도록 하고
     - 마지막으로 va해서 히든사이즈 투 1
- 스커어
    - 주어진 라스트 히든과 배치니까 일단 닷하려면 언스퀴즈해서 배치사이즈, 히든 사이즈1되고,
    - 시쿼늣 연산은 어탠션은 배치사이즈, 맥스, 히든과 쿼리 백터인 배치사이즈 히든딤이랑 연ㅅ나해서 비교해주면 되는거죠
    - 어떤 특정한 결과값으로 딱 맥스 타임 스탭 1의 결과가 나오고결과에 겹치는거 사라져서 나와 
    - 마지막 1 없애면 배치에 그거랑 가탕. 
    - 앞에 있는게 인코더 히든이니까 
    - return encoder_output. 
    - 라스트 히든 스테이터스.. 결론적으로 batchsize, maxtimestep이 됩니다. 
- 제너럴
    - 더블유 에이 곱해주고. 
    - 마지막에 배치사이즈 히든에서 일을 더해서 닷프로덕트 똑같이 하기 위해 언스퀴
    - return (encoder_output)
-콘켓
    - 현재 타겟과 인코더를 concat해주고 탄에이치 고치고 트랜스포스 하면 됩니다
     - 인코더 아웃풋과 사이즈를 맞춰야합니다. 인코더 아웃풋은 (b mat time. hidden_state)인데 (b.hidden_state). 
        - 처음으로 언스퀴즈 해줘서 우리가 (b,1,hiddensize)맞추고 복사해서 maxtimesize만큼 복사를 해주면 됩니다. 
        - 라스트 히든에 언스퀴즈를 하면ㄷ ㅚ고, 해당되는 크기를 인코더 아웃풋만큼 익스팬드하고 . 
        - 우리가 엑스가 이 사이즈가 되니까 원래이 인풋인 엑스와쿼리와콘켓을 하는데 이 콘켓은 hiddenstate로 하는데, -1이라는 거로 해.......? @@@@@@@
        - 그 다음에 해준거를 우리가 리니어 트랜스포션으로 transpose> tanh 거치고 
        - 마지막으로 구해진게 현재 배치사이즈 맥스 히든 등이니까이걸 배치 사이즈 맥스타임 1로 바꾸고, 1을 날려서 squeeze(-1)> 해줘야지 (VA?)

In [0]:
class Attention(nn.Module):
    """
    Inputs:
        last_hidden: (batch_size, hidden_size)
        encoder_outputs: (batch_size, max_time, hidden_size)
    Returns:
        attention_weights: (batch_size, max_time)
    """
    def __init__(self, batch_size, hidden_size, method="dot"):
        super(Attention, self).__init__()
        self.method = method
        self.hidden_size = hidden_size
        if method == 'dot':
            pass
        elif method == 'general':
            # Wa (hidden,hidden)
            self.Wa = nn.Linear(hidden_size, hidden_size, bias=False)
        elif method == "concat":
            # Wa : (2*hidden,hidden)
            # Va : (hidden,1)
            self.Wa = nn.Linear(2*hidden_size, hidden_size, bias=False)
            self.va = nn.Parameter(torch.FloatTensor(hidden_size, 1))
        elif method == 'bahdanau':
            # Wa : (hidden_size,hidden_size) 
            # Ua : (hidden_size,hidden_size)
            # Va : (hidden_size,1)
            self.Wa = nn.Linear(hidden_size, hidden_size, bias=False)
            self.Ua = nn.Linear(hidden_size, hidden_size, bias=False)
            self.va = nn.Parameter(torch.FloatTensor(hidden_size, 1))
        else:
            raise NotImplementedError

        
    def forward(self, last_hidden, encoder_outputs, seq_len=None):
        """
        Inputs :
          last_hidden : (B,T,hidden_size)
          encoder_outputs : 
          seq_len:  
        Returns:
          attention matrix : 
        """
        batch_size, seq_lens, _ = encoder_outputs.size()
        # attention energies 를 구하기 
        attention_energies = self._score(last_hidden, encoder_outputs, self.method)
        
        if seq_len is not None:
            attention_energies = mask_3d(attention_energies, seq_len, -float('inf'))

        return F.softmax(attention_energies, -1)

    def _score(self, last_hidden, encoder_outputs, method):
        """
        Computes an attention score
        :param last_hidden: (batch_size, hidden_dim)
        :param encoder_outputs: (batch_size, max_time, hidden_dim)
        :param method: str (`dot`, `general`, `concat`)
        :return:
        """

        # assert last_hidden.size() == torch.Size([batch_size, self.hidden_size]), last_hidden.size()
        
        if method == 'dot':
            last_hidden = last_hidden.unsqueeze(-1) # (batch_size, hidden_dim,1)
            
            # attention : (batch_size,max_time, hidden_dim) , (batch_size,hidden_dim,1) - > (batch_size,max_time ,1)
            
            return ???.bmm(last_hidden).squeeze(-1)  

        elif method == 'general':
            # dot 이랑 비슷 다만 last hidden을 한번 projection
            x = self.??(last_hidden) # (batch_size, hidden_dim) ->  (batch_size, hidden_dim)
            x = x.unsqueeze(-1) # (batch_size, hidden_dim) ->  (batch_size, hidden_dim,1)
            # encoded 된 hidden states 와 dot proudct를 수행하기 
            # attention: (batch_size,max_time, hidden_dim) , (batch_size,hidden_dim,1) - > (batch_size,max_time ,1)
            return ???.bmm(x).squeeze(-1)

        elif method == "concat":
            x = last_hidden.unsqueeze(1).expand_as(encoder_outputs) # (batch_size, hidden_dim) ->  (batch_size,1, hidden_dim)
            # concat 후 -> linear 거치기 -> 후 tanh
            x = F.??(self.??(torch.??((x, encoder_outputs), -1))) # (batch_size, max_timestep, hidden_dim) ->  (batch_size,  max_timestep, hidden_dim*2)
            # (batch_size, max_timestep, hidden_dim*2) ->  (batch_size,  max_timestep, )
            return x.matmul(self.??).squeeze(-1)

        elif method == "bahdanau":
            # mlp 기반의 attention model
            
            x = last_hidden.unsqueeze(1) # (batch_size, hidden_dim) ->  (batch_size,1, hidden_dim)
            # 각각을 projection 후 더하기 -> tanh 
            out = F.???(self.??(x) ?? self.???(encoder_outputs)) # 
            return ??.matmul(self.va).squeeze(-1)# (batch_size,max_timestep,hidden_dim) ->  (batch_size, max_timestep)

        else:
            raise NotImplementedError

### Seq2Seq Model 

# 여태까지 해온 구조를 그대로 따라하면 됩니다.
- 초기 히든 스테이터스를 만들어야 할거 같은데
- 아까 인코더에 인잇이 있어서 그거 통해서 초창기 만ㄷ르고 배치 사이즈 만큼 만들어줍니다.
- 그 이후에 인코더 족에 집어넣으면 모델이 돌아가게 됩니다.
- 이노더를 넣어주고 인코더에는 원래 인풋과 인잇과 랭스를 넣어서 이노드가 되고
- 디코드는 디코더에서 만들어진 인코더 히든 및 가각 타임스탬프를 가지고 디코드를해요
- 초창기에는 배치 사이즈가 1이거나 첫번쨰 타임스탬프에서는 토큰이 없으면 넣어야 하니 스타트 토큰을 너허야해 sos. 
-그래서 디코더 초창기 인풋 넣고 그 밑은 계산하는 수식이라서 step, loss는 넘어갈게요.
- 인코드 시퀀스 시퀀스 주요한 함수
### stemp
    - 배치를 엑스와 와이(소스와 타겟), 엑스 랭쓰와 와이 랭써로 바꾸고 
    - gpu로 쿠다에 올립니다
    - 인코더로 해야하니까 이코더 함수로 진행을 해주고요
    - 그 다음에 인코더로 진행한 애를 디코드를 진행하비다. 
    - 디코드 함수와 인코더 아웃풋 인코더 스테이터스 넣을 와이.
    - 각각 로짓과 레이블이 나옵니다. 이거 스탭으로 알아내고
    - 추가적으로 로짓을 로스 펑션으로 구하는데 이 로스 펑셩는 fn(logits, labes)비교해서 구할 수 있습니다
    - 디코드는 너무 길어서 옆에있는 주석 보는게 좋아요. 간단히 설명하면 이노더가 인코딩 용이고 인코더를 초창기 히든으로 하고 포워드하고, 디코드는인코드 지난 후 히든 스테이터스 바탕으로 랭쓰가 있는데 이건 디코더 부분에서는 각 타임스탬프마다 하나씩 넣으며 돌아가서 그만큼 이터레이터 돌고. 어텐션 하깅 위한 인풋 넣고 프리비어스 히든 스테이터스 넣어줍니다. 
        - 다만 셀프점 트레이닝과 아닌 경우가 이써. 트레이닝인경우 정답을 넣게하느ㅏㄴ거고. 여기서 엘스이면 가장 높은 값만 쓰게 했어요. 이게 시퀀스시퀀스 구조

![대체 텍스트](https://i.ibb.co/CK5wTz5/crossentropy.png)

![대체 텍스트](https://i.ibb.co/ssXZ28q/crossentropy-2.png)

In [0]:
class Seq2Seq(nn.Module):
    """
        Sequence to sequence module
    """

    def __init__(self, config):
        super(Seq2Seq, self).__init__()
        self.SOS = config.get("start_index", 1) # Start index를 가져옵니다. 
        self.vocab_size = config.get("n_classes", 32) # embedding 에 필요한 vocabulary size 
        self.batch_size = config.get("batch_size", 1) # batch_size 정보를 가져옵니다.
        self.gpu = config.get("gpu", False) # cuda 로 돌아가는지 아닌지에 대한 정보 

        # Encoder 선언
        
        self.encoder = EncoderRNN(config)

        # Decoder 선언 
        
        self.decoder = BahdanauDecoder(config)
        
        # loss fucntion 
        # ignore_index =0 왜???
        self.loss_fn = torch.nn.CrossEntropyLoss(ignore_index=0)
        
        

    def encode(self, x, x_len):
        # encoder를 통해 주어진 source 정보를 Encodeing 하는 용도 
        
        batch_size = x.size()[0]
        # 초기 inital hidden state 만들기
        init_state = self.encoder.???_hidden(batch_size)
        # encoder Forward 수행 
        encoder_outputs, encoder_state = self.???.forward(x, init_state, x_len)
        
        
       
        return encoder_outputs, encoder_state.squeeze(0)

    def decode(self, encoder_outputs, encoder_hidden, targets, targets_lengths, input_lengths):
        """
        Args:
            encoder_outputs: (B, T, H)
            encoder_hidden: (B, H)
            targets: (B, L)
            targets_lengths: (B)
            input_lengths: (B)
        Vars:
            decoder_input: (B)
            decoder_context: (B, H)
            hidden_state: (B, H)
            attention_weights: (B, T)
        Outputs:
            alignments: (L, T, B)
            logits: (B*L, V)
            labels: (B*L)
        """

        batch_size = encoder_outputs.size()[0]
        max_length = targets.size()[1]
        # decoder의 처음 y0 는 무엇이 되어야 할까? *주의해야할 포인트 
        if batch_size ==1:
          decoder_input = torch.LongTensor([self.???] * batch_size)
        else:
          decoder_input = torch.LongTensor([self.???] * batch_size).squeeze(-1)
        decoder_context = encoder_outputs.transpose(1, 0)[-1] #(Batch,1)
        decoder_hidden = encoder_hidden
        
        #alignments :  attention align을 저장하기 위한 용도  
        alignments = torch.zeros(max_length, encoder_outputs.size(1), batch_size) # attention align을 저장하기 위한 용도 
        logits = torch.zeros(max_length, batch_size, self.decoder.output_size) # logits 값을 저장하기 위한 용도의 tensor 

        if self.gpu:
            decoder_input = decoder_input.cuda()
            decoder_context = decoder_context.cuda()
            logits = logits.cuda()
        inference = []
        for t in range(max_length):

            # The decoder accepts, at each time step t :
            # - an input, [B]
            # - a context, [B, H]
            # - an hidden state, [B, H]
            # - encoder outputs, [B, T, H]
            
            # The decoder outputs, at each time step t :
            # - an output, [B]
            # - a context, [B, H]
            # - an hidden state, [B, H]
            # - weights, [B, T]

            outputs, decoder_hidden, attention_weights = self.decoder.forward(
                    input=decoder_input.long(),
                    prev_hidden=decoder_hidden,
                    encoder_outputs=encoder_outputs,
                    seq_len=input_lengths)
            
            alignments[t] = attention_weights.transpose(1, 0)
            
            
            logits[t] = outputs

            

            if  self.training:
                decoder_input = targets[:, t]
            else:
                topv, topi = outputs.data.topk(1) # 가장 높은 예측만 사용.
                decoder_input = topi.squeeze(-1).detach()
                inference.append(decoder_input.cpu())

        
        labels = targets.contiguous().view(-1) 

        
        mask_value = 0
        #what is this mask_3d? # (warning check)
        logits = mask_3d(logits.transpose(1, 0), targets_lengths, mask_value)
        logits = logits.contiguous().view(-1, self.vocab_size) # loss를 구하기 위해 쫙 펴주기 

        return logits, labels.long(), alignments,inference

    
    def step(self, batch):
        x, y, x_len, y_len = batch
        if self.gpu:
            x = x.cuda()
            y = y.cuda()
            x_len = x_len.cuda()
            y_len = y_len.cuda()

        encoder_out, encoder_state = self.???(x, x_len) # encoder 
        logits, labels, alignments,inference = self.???(encoder_out, encoder_state, y, y_len, x_len) # decoder 를 통해 alignment와 logit 값 얻기 
        return logits, labels, alignments,inference

    def loss(self, batch):
        logits, labels, alignments,inference = self.step(batch)
        loss = self.loss_fn(????, labels) # loss 구하기 우리는 cross entropy 사용 
        return loss, logits, labels, alignments,inference

### Train the model

![대체 텍스트](https://pytorch.org/tutorials/_images/seq2seq.png)

- 트레인> 배치뽑고> 포워드 진행을 하고> 포워드 진행은 로스 펑션> 스탭에서 인코더 드코드하고 로스값구하는게 로스> 그 다음에 그래디ㅓㄴ트 구하기 전에 옵티마이저에 버퍼 비워주고> 이걸 비웠으니까 백프로파해서 로스를 백워드 해주게 되면 그래디언트 계싼> 오비마이저 스탭을 하면 그래디언트 기반으로 업데이트

- 만일 클립그래디언트 하고 싶다면 그냥
    - torch.nn.utils.clip_grad_norms(model.parameters(), max_norm=2 )모델이 2가 되지 않는 선에서 해주겠다는거. 
    
    
    # eval
    - eval에서 드랍아웃
    - ** with torch.no_grad()
        - 밑에있는 연산을 할때 그래디언트위해 어떤 메모리 저장하지 말라 말해
        - 밑에서 메모리 사용량 및 속도 안 느려지게
        - 우리가 백프로하기 위해선느 각 레이어 아웃풋 저장해야하는데 그럼 느려져서 그런 부분 방지코자

In [0]:
def train(model, optimizer, train_loader, epoch,n_epochs):
    

    losses = []
    cers = []

    
    model.train() # train mode 
    count = 0
    for batch in train_loader:
        loss, _, _, _,_ = model.loss(batch)
        losses.append(loss.item())
        # Reset gradients
        optimizer.????()
        # Compute gradients
        loss.???()
        # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=2)
        optimizer.???()
  
    print ('\n [{}/{}] avg_loss= {:05.3f}'.format(epoch,n_epochs,np.mean(losses)))
    
    return model, optimizer

In [0]:
def evaluate(model, eval_loader):

    losses = []
    accs = []
    edits = []
    
    model.eval() # why?? 

    with torch.no_grad():
        for batch in eval_loader:
            #t.set_description(" Evaluating... (train={})".format(model.training))
            loss, logits, labels, alignments,_ = model.loss(batch)
            preds = logits.detach().cpu().numpy()
            
            acc = 100 *np.sum(np.argmax(preds, -1) == labels.detach().cpu().numpy()) / len(preds)
            edit = editdistance.eval(np.argmax(preds, -1), labels.detach().cpu().numpy()) / len(preds)
            
            losses.append(loss.item())
            
            accs.append(acc)
            edits.append(edit)
        
        

   
    print("  End of evaluation : loss {:05.3f} , acc {:03.1f} , edits {:03.3f}".format(np.mean(losses), np.mean(accs), np.mean(edits)))
    


## 학습을 진행해보도록 하겠습니다

In [0]:
USE_CUDA = torch.cuda.is_available()
batch_size = 32
epochs = 6
dataset = ToyDataset(5, 15)
eval_dataset = ToyDataset(5, 15, type='eval')

In [0]:
train_loader = data.DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=pad_collate, drop_last=True) #셔플 트루로 변경
eval_loader = data.DataLoader(eval_dataset, batch_size=batch_size, shuffle=False, collate_fn=pad_collate,drop_last=True)

In [0]:
config = {
  "decoder": "Bahdanau",
  "encoder": "RNN",
  "n_channels": 4,
  "encoder_hidden": 64,
  "encoder_layers": 1,
  "encoder_dropout": 0.2,
  "bidirectional_encoder": False,
  "decoder_hidden": 64,
  "decoder_layers": 1,
  "decoder_dropout": 0.2,
  "n_classes":dataset.VOCAB_SIZE+3 ,
  "batch_size": 32,
  "embedding_dim": 64,
  "attention_score": "concat",
  "learning_rate": 0.001,
  "gpu": True,
  "loss": "cross_entropy"
}

In [0]:
model = Seq2Seq(config) #아까 a,b,c,d,e 해쓰니 이거 갯수 맞는지 체크
model = model.???()

In [0]:
optimizer = torch.optim.Adam(model.????(), lr=config.get("learning_rate", .001))

In [0]:
for epoch in range(epochs):
  model,optimizer  = train(model,optimizer, train_loader,epoch,epochs)
  evaluate(model,eval_loader)
 

### 시각화

In [0]:
import seaborn

def draw(data, x, y):
    seaborn.heatmap(data, 
                    xticklabels=x, square=True, yticklabels=y, vmin=0.0, vmax=1.0, 
                    cbar=False)

In [0]:
def visualize_plot(model,custom_input= 'cgdafa'):
    c_xs = np.array([dataset.char2int.get(x) for x in custom_input]+[2])
    c_xs = torch.from_numpy(c_xs).unsqueeze(0).long()

    c_xl = torch.tensor(c_xs[0].size()[-1]).unsqueeze(0)

    c_ys = np.array([dataset.char2int.get(x) for x in custom_input[::-1]] + [2]) # Return the random string and its reverse + EOS 
    c_ys = torch.from_numpy(c_ys).unsqueeze(0).long()

    c_yl = torch.tensor(c_ys[0].size()[-1]).unsqueeze(0)
    c_data = (c_xs,c_ys,c_xl,c_yl)
    loss, logits, labels, alignments,predict=model.loss(c_data)
    heat_map_value = alignments.detach().cpu().numpy()[:, :, 0]
    preds = logits.detach().cpu().numpy()
    preds = np.argmax(preds, -1)
    source_tokens = [ dataset.int2char[item-3] for item in c_xs[0] if item!=0 if item !=2 ] +['</s>']
    target_tokens = [ dataset.int2char[item-3] if item !=2 else '</s>' for item in preds.tolist() if item!=0 ]
    draw(heat_map_value,source_tokens,target_tokens)
    

In [0]:
visualize_plot(model,'cbada')

### Dot Mode 

In [0]:
config = {
  "decoder": "Bahdanau",
  "encoder": "RNN",
  "n_channels": 5,#아까 a b c d e  5개 했으니까
  "encoder_hidden": 64,
  "encoder_layers": 1,
  "encoder_dropout": 0.2,
  "bidirectional_encoder": False,
  "decoder_hidden": 64,
  "decoder_layers": 1,
  "decoder_dropout": 0.2,
  "n_classes":dataset.VOCAB_SIZE+3 ,
  "batch_size": 32,
  "embedding_dim": 64,
  "attention_score": "concat",
  "learning_rate": 0.001,
  "gpu": True,
  "loss": "cross_entropy"
}

In [0]:
model = Seq2Seq(config)
model = model.cuda()

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=config.get("learning_rate", .001))

In [0]:
for epoch in range(epochs):
  model,optimizer  = train(model,optimizer, train_loader,epoch,epochs) #아담으로 할게요 
  evaluate(model,eval_loader)
 

In [0]:
visualize_plot(model,'cbada')

### Concat Mode 

In [0]:
config = {
  "decoder": "Bahdanau",
  "encoder": "RNN",
  "n_channels": 4,
  "encoder_hidden": 64,
  "encoder_layers": 1,
  "encoder_dropout": 0.2,
  "bidirectional_encoder": False,
  "decoder_hidden": 64,
  "decoder_layers": 1,
  "decoder_dropout": 0.2,
  "n_classes":dataset.VOCAB_SIZE+3 ,
  "batch_size": 32,
  "embedding_dim": 64,
  "attention_score": "concat",
  "learning_rate": 0.001,
  "gpu": True,
  "loss": "cross_entropy"
}

In [0]:
model = Seq2Seq(config)
model = model.cuda()

In [0]:
optimizer = torch.optim.Adam(model.parameters(), lr=config.get("learning_rate", .001))

In [0]:
for epoch in range(epochs):
  model,optimizer  = train(model,optimizer, train_loader,epoch,epochs)
  evaluate(model,eval_loader)
 

In [0]:
visualize_plot(model,'cbada')